In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
print("at directory:", os.getcwd())
print("changing to root directory")
os.chdir("../../..")
print("at directory:", os.getcwd())

at directory: /Users/wilka/git/research/jaxneurorl/projects/humansf/analysis
changing to root directory
at directory: /Users/wilka/git/research/jaxneurorl


In [4]:
import jax
import jax.numpy as jnp
import functools
from functools import partial
import pandas as pd
import numpy as np
from pprint import pprint
import glob
from glob import glob
import matplotlib.pyplot as plt
import polars as pl
from tqdm.notebook import tqdm

from projects.humansf import data_loading
from projects.humansf import housemaze_analysis
from projects.humansf import data_loading
from projects.humansf import housemaze_trainer
from projects.humansf import housemaze_experiments
from projects.humansf import networks
from housemaze.human_dyna import mazes

from housemaze import renderer
from housemaze import utils
from housemaze.human_dyna import multitask_env as maze
from jaxneurorl import launcher
from jaxneurorl.agents import value_based_basics as vbb


from projects.humansf import download_data

No file specified for image dict.
Using: /Users/wilka/git/research/jaxneurorl/libraries/housemaze/housemaze/image_data.pkl


wandb: WARNING `wandb.require('core')` is redundant as it is now the default behavior.


No file specified for image dict.
Using: /Users/wilka/git/research/jaxneurorl/libraries/housemaze/housemaze/image_data.pkl
No file specified for groups.
Using: /Users/wilka/git/research/jaxneurorl/libraries/housemaze/housemaze/human_dyna/list_of_groups.npy


/Users/wilka/miniforge3/envs/jaxneurorl/lib/python3.10/site-packages/jax/_src/numpy/array_methods.py:118: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  return lax_numpy.astype(self, dtype, copy=copy, device=device)


In [5]:
# !ls -lhtr ~/git/research/results/human_dyna/user_data/*

In [6]:
# !rm ~/git/research/results/human_dyna/user_data/*3714894224_name* ~/git/research/results/human_dyna/user_data/*2631242498_name*

In [9]:
from projects.humansf.download_data import download_user_files

bucket_name = "human-dyna"
prefix = "data/"
#pattern = "data/data_user=*_name=r0-v2*debug=0.json"
human_data_pattern = "data/data_user=*exp=3*debug=0.json"
destination_folder = "/Users/wilka/git/research/results/human_dyna/user_data/exp3"

download_user_files(bucket_name, prefix, human_data_pattern, destination_folder)

Downloaded: data/data_user=1004981584_name=exp_exp=3_debug=0.json to /Users/wilka/git/research/results/human_dyna/user_data/exp3/data_user=1004981584_name=exp_exp=3_debug=0.json
Downloaded: data/data_user=1300029984_name=exp_exp=3_debug=0.json to /Users/wilka/git/research/results/human_dyna/user_data/exp3/data_user=1300029984_name=exp_exp=3_debug=0.json
Downloaded: data/data_user=1397338607_name=exp_exp=3_debug=0.json to /Users/wilka/git/research/results/human_dyna/user_data/exp3/data_user=1397338607_name=exp_exp=3_debug=0.json
Downloaded: data/data_user=1648586841_name=exp_exp=3_debug=0.json to /Users/wilka/git/research/results/human_dyna/user_data/exp3/data_user=1648586841_name=exp_exp=3_debug=0.json
Downloaded: data/data_user=1678765622_name=exp_exp=3_debug=0.json to /Users/wilka/git/research/results/human_dyna/user_data/exp3/data_user=1678765622_name=exp_exp=3_debug=0.json
Downloaded: data/data_user=1920467653_name=exp_exp=3_debug=0.json to /Users/wilka/git/research/results/human_d

In [22]:
data_dir = '/Users/wilka/git/research/results/human_dyna/'
files = f'{data_dir}/user_data/*exp3*/*exp=3*.json'
user_base_path = f'{data_dir}/user_data'
files = list(set(glob(files)))
len(files)

26

In [23]:
import json
nfinished = 0
for idx, file in enumerate(files):
    with open(file, 'r') as f:
        try:
            data = json.load(f)
        except Exception as e:
            print('-'*25)
            print(idx, os.path.basename(file))
            print(e)
            continue
        finished = data[-1].get("finished", False)
        if finished:
            print
            nfinished += 1
            print('-'*25)
            print(idx, os.path.basename(file))
            print("\n" + data[-1]['feedback'] or "N/A")
            print('bonus:', data[-1]['bonus'])
            print('feedback:')
            pprint(data[-2]['data'])

        # else:
        #     print("NOT FINISHED")

-------------------------
0 data_user=3333226705_name=exp_exp=3_debug=0.json

There was a slight confusion on my part when both the continue and start buttons appeared on the screen near the end of the last part. I clicked on the continue button but nothing seem to happen and then I clicked on the start button which took me to the screen to find the object in the maze.
bonus: 3
feedback:
{'feedback': 'I thought the path I took was the fastest and I quickly decided '
             'on it.',
 'noticed_path': 3}
-------------------------
1 data_user=472713048_name=exp_exp=3_debug=0.json

This was too tedious. Too many phases. Make it shorter. 
bonus: 3
feedback:
{'feedback': 'I felt like going the long way around. I also did that once for '
             'the knife, but then had to turn around. I was getting bored and '
             'wanted to switch things up.',
 'noticed_path': 3}
-------------------------
2 data_user=2030323034_name=exp_exp=3_debug=0.json

Great
bonus: 3
feedback:
{'feed

In [18]:
from pprint import pprint
pprint(data[-1])
pprint(data[-2])

{'bonus': 3,
 'feedback': 'It felt quite unresponsive for some reason.  Perhaps that was '
             "intentional for your purposes, but just in case it wasn't, it "
             'was sort of like playing a video game with extremely high '
             'latency.  In general I was using a lot more inputs than should '
             'have been necessary and just kind of bashing on the keys the '
             'amount that felt right to get to where I needed to go.\n'
             '\n'
             "It did seem like some of the times I reached the goal it didn't "
             "count properly for some reason too, though I'm not sure if that "
             'was due to something not counting correctly or if I simply '
             "wasn't where I assumed I was due to the unresponsiveness.",
 'finished': True}
{'data': {'feedback': "Once I found a reasonable seeming path I didn't even "
                      "really consider looking for others.  It didn't seem "
                      'like 

In [24]:
data[-3]

{'id': 3271,
 'session_id': '204ada23-39ff-4cb7-a919-29887ccabea6',
 'name': 'big_m4_maze_short_eval_diff',
 'stage_idx': 23,
 'data': {'image_seen_time': '2024-10-12T18:27:07.646Z',
  'action_taken_time': '2024-10-12T18:27:07.646Z',
  'computer_interaction': 'timer',
  'action_name': 'timer',
  'action_idx': -1,
  'timelimit': 5,
  'timestep': 'hqVzdGF0ZY2ja2V5gqEwzmVhpJ2hMc5+a7cxqHN0ZXBfbnVtAqRncmlk3gASoTDeABqhMIGhMAChMYGhMAChMoGhMAChM4GhMAChNIGhMAChNYGhMAGhNoGhMAChN4GhMAChOIGhMAChOYGhMAGiMTCBoTAAojExgaEwAaIxMoGhMACiMTOBoTABojE0gaEwAKIxNYGhMACiMTaBoTABojE3gaEwAKIxOIGhMACiMTmBoTABojIwgaEwAaIyMYGhMACiMjKBoTAAojIzgaEwAKIyNIGhMAGiMjWBoTAAoTHeABqhMIGhMAChMYGhMAGhMoGhMAChM4GhMAGhNIGhMAChNYGhMAGhNoGhMAChN4GhMAGhOIGhMAChOYGhMAGiMTCBoTArojExgaEwAaIxMoGhMB2iMTOBoTABojE0gaEwAKIxNYGhMAGiMTaBoTABojE3gaEwAKIxOIGhMAGiMTmBoTABojIwgaEwAaIyMYGhMAGiMjKBoTABojIzgaEwAKIyNIGhMAGiMjWBoTAAoTLeABqhMIGhMAChMYGhMAGhMoGhMAChM4GhMAGhNIGhMAChNYGhMAGhNoGhMAChN4GhMAGhOIGhMAChOYGhMAGiMTCBoTAAojExgaEwAaIxMoGhMACiMTOB

In [59]:
user_df, user_data = data_loading.make_all_episode_data(
    files=files,
    example_timestep=example_timestep,
    base_path=user_base_path,
    overwrite=True,
)

Processing files:   0%|          | 0/2 [00:00<?, ?it/s]

Filtered 46 data point


Processing episodes:   0%|          | 0/51 [00:00<?, ?it/s]

Filtered 65 data point


Processing episodes:   0%|          | 0/49 [00:00<?, ?it/s]